<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Note: For viewing the jupyter notebook with all the interactive elements, use https://nbviewer.jupyter.org/ to view this  jupyter notebook. As On GitHub, they are displayed as static HTML sites. 

## Introduction

In this notebook, we converted addresses into their equivalent latitude and longitude values. Also, we have used the Foursquare API to explore neighborhoods in Toronto. We have used the **explore** function to get the most common venue categories in each neighborhood, and then used this feature to group the neighborhoods into clusters. We have used the *k*-means clustering algorithm to complete the clustering process. Finally, we have used the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [1]:
# Importing necessary libraries

import pandas as pd # library to handle data in a vectorized manner

import numpy as np # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors 

import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from sklearn.cluster import KMeans # import k-means from clustering stage

## 1. Download and Explore Dataset

*Here we will be using read_html function of the pandas library. Using this function we pass in the url of webpage where our data is present. Now the read_html function goes through the webpage and if tables are present on the webpage it will return it in the form of list contaning all the tables that were present. Since we only have one table, so we use list indexing to access it. The tables are returned or stored as dataframe in the list.*

In [2]:
# Loading the data present on the html page
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print('Data is loaded!')

Data is loaded!


*The dataframe  consists of three columns: PostalCode, Borough, and Neighborhood.*

In [3]:
# Let's take a quick look at the data
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


*We are going to only process the cells that have an assigned borough. We will ignore cells with a borough that is 'Not assigned'.*

In [4]:
# Checking the boroughs that are 'Not assigned'
df['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [5]:
# Subset the dataframe to access boroughs, not including 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# Checking the boroughs in the dataframe
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.*

In [7]:
# Checking if a cell has a borough but a 'Not assigned' neighborhood
(df['Neighbourhood'] == 'Not assigned').unique()

array([False])

*So we don't have any neighbourhood that is 'Not assigned'*

In [8]:
# Number of rows and columns in dataframe
df.shape

(103, 3)

## 2. Getting latitude and the longitude coordinates of each neighborhood.

*Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.*

*Importing csv file containing the latitude and longitude values of postal codes.*

In [9]:
geo_data = pd.read_csv(r'C:\Users\rajpu\Desktop\Canada Clustering\Geospatial_Coordinates.csv')

In [10]:
# Let's take a quick look at the data
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
# Merging the two dataframes
result = pd.merge(df,geo_data,on = 'Postal Code')

In [12]:
# Let's take a quick look at the data
result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


*Use geopy library to get the latitude and longitude values of Toronto.*

*In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ca_explorer</em>, as shown below.*

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


*Create a map of Toronto with neighborhoods superimposed on top.*

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, postal in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood'],result['Postal Code']):
    label = '{}, {} ,{} '.format(neighborhood, borough, postal)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

*Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.*

In [15]:
result['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

*Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.*

### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'PPMPJALBRPPZYBGOHCYXLT502R2BLBYPEZUK1HHBU1HQLJVO' # your Foursquare ID
CLIENT_SECRET = 'L4F3S0C1TEESUHW3C1X4PNOYB3FHFBSLDBWFRRUJEIN1DSTN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PPMPJALBRPPZYBGOHCYXLT502R2BLBYPEZUK1HHBU1HQLJVO
CLIENT_SECRET:L4F3S0C1TEESUHW3C1X4PNOYB3FHFBSLDBWFRRUJEIN1DSTN


### Let's explore the first neighborhood in our dataframe.

*Get the neighborhood's name.*

In [17]:
result.loc[0, 'Neighbourhood']

'Parkwoods'

*Get the neighborhood's latitude and longitude values.*

In [18]:
neighborhood_latitude = result.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = result.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = result.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

*First, let's create the GET request URL. Name your URL 'url'.*

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=PPMPJALBRPPZYBGOHCYXLT502R2BLBYPEZUK1HHBU1HQLJVO&client_secret=L4F3S0C1TEESUHW3C1X4PNOYB3FHFBSLDBWFRRUJEIN1DSTN&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

*Send the GET request and examine the results*

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3331d711597434c51919ac'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

*From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.*

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

*Now we are ready to clean the json and structure it into a pandas dataframe.*

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\rajpu\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


*And how many venues were returned by Foursquare?*

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [25]:
toronto_venues = getNearbyVenues(names = result['Neighbourhood'],
                                   latitudes = result['Latitude'],
                                   longitudes = result['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Let's check the size of the resulting dataframe

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(2131, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


*Let's check how many venues were returned for each neighborhood*

In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",14,14,14,14,14,14


### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


## 3. Analyze Each Neighborhood

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# adding values from toronto_venues neighborhood column to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# Column position of 'Neighborhood'
a = toronto_onehot.columns.tolist().index('Neighborhood')
print("Index position of 'Neighborhood' column:",a)

# move neighborhood column to the first column
first = toronto_onehot.columns.tolist()[:a]
second = toronto_onehot.columns.tolist()[a:]
fixed_columns = second + first
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Index position of 'Neighborhood' column: 189


,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon
0,Parkwoods,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

*And let's examine the new dataframe size.*

In [30]:
toronto_onehot.shape

(2131, 269)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000

#### Let's confirm the new size

In [32]:
toronto_grouped.shape

(96, 269)

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1               Skating Rink  0.25
2                     Lounge  0.25
3  Latin American Restaurant  0.25
4             Farmers Market  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1            Pool  0.14
2             Pub  0.14
3     Coffee Shop  0.14
4  Sandwich Place  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Coffee Shop  0.09
1               Bank  0.09
2  Convenience Store  0.05
3          Gift Shop  0.05
4        Gas Station  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2  Japanese Restaurant  0.25
3                 Bank  0.25
4        Garden Center  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2      Sandwich Place  0.08
3          Restaura

#### Let's put that into a pandas dataframe

*First, let's write a function to sort the venues in descending order.*

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

*Now let's create the new dataframe and display the top 10 venues for each neighborhood.*

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Aquarium,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Nail Salon
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Coffee Shop,Gym,Pool,Nail Salon,Aquarium,Airport Terminal,American Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Pharmacy,Gas Station,Gift Shop,Sushi Restaurant,Grocery Store,Supermarket,Diner
3,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Art Gallery,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Butcher,Pizza Place,Indian Restaurant,American Restaurant,Juice Bar,Pub


## 4. Cluster Neighborhoods

*Run k-means to cluster the neighborhood into 6 clusters.*

In [36]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

*Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.*

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.columns = ['Cluster Labels', 'Neighbourhood', '1st Most Common Venue','2nd Most Common Venue', '3rd Most Common Venue','4th Most Common Venue', '5th Most Common Venue','6th Most Common Venue', '7th Most Common Venue','8th Most Common Venue', '9th Most Common Venue','10th Most Common Venue']

toronto_merged = result

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Food & Drink Shop,Park,Nail Salon,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Nail Salon,Portuguese Restaurant,Hockey Arena,Grocery Store,Coffee Shop,Antique Shop,Airport Service,Airport Terminal,American Restaurant,Aquarium
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Gym / Fitness Center,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Boutique,Accessories Store,Vietnamese Restaurant,Coffee Shop,Event Space,Aquarium,Airport Service,Airport Terminal
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Diner,Theater,Yoga Studio,Italian Restaurant,Sandwich Place,Smoothie Shop,Hobby Shop,Gym,Sushi Restaurant


In [38]:
# Checking for any null value
print(toronto_merged.isnull().sum())

# Dropping the null values
toronto_merged.dropna(axis = 0,inplace = True)

Postal Code               0
Borough                   0
Neighbourhood             0
Latitude                  0
Longitude                 0
Cluster Labels            3
1st Most Common Venue     3
2nd Most Common Venue     3
3rd Most Common Venue     3
4th Most Common Venue     3
5th Most Common Venue     3
6th Most Common Venue     3
7th Most Common Venue     3
8th Most Common Venue     3
9th Most Common Venue     3
10th Most Common Venue    3
dtype: int64


In [39]:
# Converting 'Cluster Labels' column dtype to int
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int')

In [40]:
# Let's take a look at the DataSet
toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Food & Drink Shop,Park,Nail Salon,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Nail Salon,Portuguese Restaurant,Hockey Arena,Grocery Store,Coffee Shop,Antique Shop,Airport Service,Airport Terminal,American Restaurant,Aquarium
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Gym / Fitness Center,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Boutique,Accessories Store,Vietnamese Restaurant,Coffee Shop,Event Space,Aquarium,Airport Service,Airport Terminal
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Theater,Yoga Studio,Italian Restaurant,Sandwich Place,Smoothie Shop,Hobby Shop,Gym,Sushi Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Nail Salon,Airport Gate,BBQ Joint,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Coffee Shop,Beer Store,Restaurant,Japanese Restaurant,Clothing Store,Bike Shop,Supermarket,Italian Restaurant,Sporting Goods Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Gastropub,Athletics & Sports,Bank,Intersection,Gym / Fitness Center,Pharmacy,Breakfast Spot,Airport Terminal,Airport Lounge
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Bookstore,Theater,Electronics Store
10,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Japanese Restaurant,Asian Restaurant,Pub,Nail Salon,Art Gallery,Airport Terminal,American Restaurant,Antique Shop,Aquarium


*Finally, let's visualize the resulting clusters*

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters and Observations

*Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.*

#### Cluster 1

*Most of the neighborhoods are grouped in cluster 1 and services or venues that are most common in cluster 1 are Nail Salon, Coffe Shop , Gym, Clothing Store.*

In [42]:
cluster_1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster_1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Nail Salon,Portuguese Restaurant,Hockey Arena,Grocery Store,Coffee Shop,Antique Shop,Airport Service,Airport Terminal,American Restaurant,Aquarium
2,Downtown Toronto,0,Coffee Shop,Café,Bakery,Pub,Park,Theater,Breakfast Spot,Restaurant,Gym / Fitness Center,Electronics Store
3,North York,0,Furniture / Home Store,Clothing Store,Boutique,Accessories Store,Vietnamese Restaurant,Coffee Shop,Event Space,Aquarium,Airport Service,Airport Terminal
4,Downtown Toronto,0,Coffee Shop,Diner,Theater,Yoga Studio,Italian Restaurant,Sandwich Place,Smoothie Shop,Hobby Shop,Gym,Sushi Restaurant
7,North York,0,Gym,Coffee Shop,Beer Store,Restaurant,Japanese Restaurant,Clothing Store,Bike Shop,Supermarket,Italian Restaurant,Sporting Goods Shop
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Italian Restaurant,Bookstore,Theater,Electronics Store
10,North York,0,Pizza Place,Japanese Restaurant,Asian Restaurant,Pub,Nail Salon,Art Gallery,Airport Terminal,American Restaurant,Antique Shop,Aquarium
11,Etobicoke,0,Golf Course,Nail Salon,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Museum
12,Scarborough,0,Bar,Home Service,Nail Salon,Art Gallery,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Museum
13,North York,0,Gym,Coffee Shop,Beer Store,Restaurant,Japanese Restaurant,Clothing Store,Bike Shop,Supermarket,Italian Restaurant,Sporting Goods Shop


#### Cluster 2

*Most common venue in neighborhoods that are grouped in cluster 2 include garden.*

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,1,Garden,Nail Salon,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Museum


#### Cluster 3

*Most common venues in neighborhoods that are grouped in cluster 3 include Park, Food and Drink Shop, Playground.*

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Food & Drink Shop,Park,Nail Salon,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery
35,East York,2,Convenience Store,Park,Art Gallery,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Nail Salon,Airport Lounge
52,North York,2,Park,Nail Salon,Art Museum,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Airport Lounge
66,North York,2,Convenience Store,Park,Art Gallery,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Nail Salon,Airport Lounge
85,Scarborough,2,Playground,Park,Airport Gate,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum
91,Downtown Toronto,2,Park,Trail,Playground,Turkish Restaurant,Vegetarian / Vegan Restaurant,Bagel Shop,Baby Store,BBQ Joint,Auto Garage,Athletics & Sports


#### Cluster 4

*Most common venues in neighborhoods that are grouped in cluster 4 include Fabric Shop and Baseball Field*

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3,Fabric Shop,Baseball Field,Nail Salon,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum
101,Etobicoke,3,Baseball Field,Nail Salon,Art Museum,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store


#### Cluster 5

*Most common venues in neighborhoods that are grouped in cluster 5 include Restaurants, so it can be concluded that the neighbourhoods belonging to this cluster have several eating places.*

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Fast Food Restaurant,Nail Salon,Airport Gate,BBQ Joint,Auto Garage,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Art Museum,Art Gallery
8,East York,4,Pizza Place,Gastropub,Athletics & Sports,Bank,Intersection,Gym / Fitness Center,Pharmacy,Breakfast Spot,Airport Terminal,Airport Lounge
50,North York,4,Pizza Place,Pharmacy,Nail Salon,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum
72,North York,4,Grocery Store,Bank,Butcher,Coffee Shop,Pizza Place,Pharmacy,Nail Salon,Airport Service,Airport Terminal,American Restaurant
82,Scarborough,4,Pharmacy,Pizza Place,Bank,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Gas Station,Noodle House,Chinese Restaurant
89,Etobicoke,4,Grocery Store,Discount Store,Fried Chicken Joint,Beer Store,Pizza Place,Pharmacy,Fast Food Restaurant,Sandwich Place,Asian Restaurant,Arts & Crafts Store
90,Scarborough,4,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Bank,Coffee Shop,Breakfast Spot,Pizza Place,Pharmacy,Bubble Tea Shop,Burger Joint


#### Cluster 6

*Cluster 6 includes most of the airport's services, so it can be concluded that cluster 6 includes neighborhoods that are near airports.*

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,York,5,Convenience Store,Art Gallery,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Nail Salon,Bakery
